In [1]:
# Import necessary libraries
import os
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For OpenAI support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts
from typing import Optional, Dict, Any

# Convenience libraries for working with Neo4j inside of Google ADK
from neo4j_for_adk import graphdb

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.CRITICAL)

print("Libraries imported.")

Libraries imported.


In [2]:
# --- Define Model Constants for easier use ---
MODEL_GPT = "openai/gpt-4o"

llm = LiteLlm(model=MODEL_GPT)

# Test LLM with a direct call
print(llm.llm_client.completion(model=llm.model, 
                                messages=[{"role": "user", "content": "Are you ready?"}], 
                                tools=[]))

print("\nOpenAI is ready for use.")

ModelResponse(id='chatcmpl-CoBMmfENaKzw9sXC2B4e598XQWUXi', created=1766076128, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_deacdd5f6f', choices=[Choices(finish_reason='stop', index=0, message=Message(content="Yes, I'm ready. How can I assist you today?", role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})], usage=Usage(completion_tokens=13, prompt_tokens=27, total_tokens=40, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None, image_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='default')

OpenAI is ready for use.


In [3]:
from helper import make_agent_caller

In [4]:
# Define the hello tool 
def say_hello(person_name: str) -> dict:
    """Formats a welcome message to a named person. 

    Args:
        person_name (str): the name of the person saying hello

    Returns:
        dict: A dictionary containing the results of the query.
              Includes a 'status' key ('success' or 'error').
              If 'success', includes a 'query_result' key with an array of result rows.
              If 'error', includes an 'error_message' key.
    """
    return graphdb.send_query("RETURN 'Hello to you, ' + $person_name AS reply",
    {
        "person_name": person_name
    })

In [5]:
# Define the new goodbye tool
def say_goodbye() -> dict:
    """Provides a simple farewell message to conclude the conversation."""
    return graphdb.send_query("RETURN 'Goodbye from Cypher!' as farewell")


In [6]:
# --- Greeting Agent ---
greeting_subagent = Agent(
    model=llm,
    name="greeting_subagent_v1",
    instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting to the user. "
                "Use the 'say_hello' tool to generate the greeting. "
                "If the user provides their name, make sure to pass it to the tool. "
                "Do not engage in any other conversation or tasks.",
    description="Handles simple greetings and hellos using the 'say_hello' tool.", # Crucial for delegation
    tools=[say_hello],
)
print(f"✅ Agent '{greeting_subagent.name}' created.")


✅ Agent 'greeting_subagent_v1' created.


In [7]:
# --- Farewell Agent ---
farewell_subagent = Agent(
    # Can use the same or a different model
    model=llm, # Sticking with GPT for this example
    name="farewell_subagent_v1",
    instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message. "
                "Use the 'say_goodbye' tool when the user indicates they are leaving or ending the conversation "
                "(e.g., using words like 'bye', 'goodbye', 'thanks bye', 'see you'). "
                "Do not perform any other actions.",
    description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.", # Crucial for delegation
    tools=[say_goodbye],
)
print(f"✅ Agent '{farewell_subagent.name}' created.")

✅ Agent 'farewell_subagent_v1' created.


In [8]:
root_agent = Agent(
    name="friendly_agent_team_v1", # Give it a new version name
    model=llm,
    description="The main coordinator agent. Delegates greetings/farewells to specialists.",
    instruction="""You are the main Agent coordinating a team. Your primary responsibility is to be friendly.
 
                You have specialized sub-agents: 
                1. 'greeting_agent': Handles simple greetings like 'Hi', 'Hello'. Delegate to it for these. 
                2. 'farewell_agent': Handles simple farewells like 'Bye', 'See you'. Delegate to it for these. 

                Analyze the user's query. If it's a greeting, delegate to 'greeting_agent'. 
                If it's a farewell, delegate to 'farewell_agent'. 
                
                For anything else, respond appropriately or state you cannot handle it.
                """,
    tools=[], # No tools for the root agent
    # Key change: Link the sub-agents here!
    sub_agents=[greeting_subagent, farewell_subagent]
)


print(f"✅ Root Agent '{root_agent.name}' created with sub-agents: {[sa.name for sa in root_agent.sub_agents]}")


✅ Root Agent 'friendly_agent_team_v1' created with sub-agents: ['greeting_subagent_v1', 'farewell_subagent_v1']


In [ ]:
from helper import make_agent_caller

root_agent_caller = await make_agent_caller(root_agent)

async def run_team_conversation():
    await root_agent_caller.call(query="Hello I'm HP", verbose=True)

    await root_agent_caller.call(query="Thanks, bye!", verbose=True)

# Execute the conversation using await
await run_team_conversation()



>>> User Query: Hello I'm HP


  [Event] Author: friendly_agent_team_v1, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={
      'agent_name': 'greeting_subagent_v1'
    },
    id='call_PKZ8VRgWRybOhdQuV7MPBHDu',
    name='transfer_to_agent'
  )
)] role='model'
  [Event] Author: friendly_agent_team_v1, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='call_PKZ8VRgWRybOhdQuV7MPBHDu',
    name='transfer_to_agent',
    response={
      'result': None
    }
  )
)] role='user'
  [Event] Author: greeting_subagent_v1, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={
      'person_name': 'HP'
    },
    id='call_SdFS2xKoMRWMUYnx6fqjq7me',
    name='say_hello'
  )
)] role='model'
  [Event] Author: greeting_subagent_v1, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='call_SdFS2xKoMRWMUYnx6fqjq7me',
    name='say_hello',
    response={
      'query_result':

In [10]:
from google.adk.tools.tool_context import ToolContext

def say_hello_stateful(user_name:str, tool_context:ToolContext):
    """Says hello to the user, recording their name into state.
    
    Args:
        user_name (str): The name of the user.
    """
    tool_context.state["user_name"] = user_name
    print("\ntool_context.state['user_name']:", tool_context.state["user_name"])
    return graphdb.send_query(
        f"RETURN 'Hello to you, ' + $user_name + '.' AS reply",
    {
        "user_name": user_name
    })

In [11]:
def say_goodbye_stateful(tool_context: ToolContext) -> dict:
    """Says goodbye to the user, reading their name from state."""
    user_name = tool_context.state.get("user_name", "stranger")
    print("\ntool_context.state['user_name']:", user_name)
    return graphdb.send_query("RETURN 'Goodbye, ' + $user_name + ', nice to chat with you!' AS reply",
    {
        "user_name": user_name
    })


print("✅ State-aware 'say_hello_stateful' and 'say_goodbye_stateful' tools defined.")


✅ State-aware 'say_hello_stateful' and 'say_goodbye_stateful' tools defined.


In [12]:
# define a stateful greeting agent. the only difference is that this agent will use the stateful say_hello_stateful tool
greeting_agent_stateful = Agent(
    model=llm,
    name="greeting_agent_stateful_v1",
    instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting using the 'say_hello' tool. Do nothing else.",
    description="Handles simple greetings and hellos using the 'say_hello_stateful' tool.",
    tools=[say_hello_stateful],
)
print(f"✅ Agent '{greeting_agent_stateful.name}' redefined.")


✅ Agent 'greeting_agent_stateful_v1' redefined.


In [13]:
farewell_agent_stateful = Agent(
    model=llm,
    name="farewell_agent_stateful_v1",
    instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message using the 'say_goodbye_stateful' tool. Do not perform any other actions.",
    description="Handles simple farewells and goodbyes using the 'say_goodbye_stateful' tool.",
    tools=[say_goodbye_stateful],
)
print(f"✅ Agent '{farewell_agent_stateful.name}' redefined.")

✅ Agent 'farewell_agent_stateful_v1' redefined.


In [14]:
root_agent_stateful = Agent(
    name="friendly_team_stateful", # New version name
    model=llm,
    description="The main coordinator agent. Delegates greetings/farewells to specialists.",
    instruction="""You are the main Agent coordinating a team. Your primary responsibility is to be friendly.

                You have specialized sub-agents: 
                1. 'greeting_agent_stateful': Handles simple greetings like 'Hi', 'Hello'. Delegate to it for these. 
                2. 'farewell_agent_stateful': Handles simple farewells like 'Bye', 'See you'. Delegate to it for these. 

                Analyze the user's query. If it's a greeting, delegate to 'greeting_agent_stateful'. If it's a farewell, delegate to 'farewell_agent_stateful'. 
                
                For anything else, respond appropriately or state you cannot handle it.
                """,
        tools=[], # Still no tools for root
        sub_agents=[greeting_agent_stateful, farewell_agent_stateful], # Include sub-agents
    )

print(f"✅ Root Agent '{root_agent_stateful.name}' created using agents with stateful tools.")


✅ Root Agent 'friendly_team_stateful' created using agents with stateful tools.


In [15]:
root_stateful_caller = await make_agent_caller(root_agent_stateful)

session = await root_stateful_caller.get_session()

print(f"Initial State: {session.state}")

Initial State: {}


In [16]:
async def run_stateful_conversation():
    await root_stateful_caller.call("Hello, I'm hp!", verbose=True)

    await root_stateful_caller.call("Thanks, bye!", verbose=True)

# Execute the conversation using await in an async context (like Colab/Jupyter)
await run_stateful_conversation()

session = await root_stateful_caller.get_session()

print(f"\nFinal State: {session.state}")


>>> User Query: Hello, I'm hp!


  [Event] Author: friendly_team_stateful, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={
      'agent_name': 'greeting_agent_stateful_v1'
    },
    id='call_FdaEyJvJtJwHXEFpW7T5SZL8',
    name='transfer_to_agent'
  )
)] role='model'
  [Event] Author: friendly_team_stateful, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='call_FdaEyJvJtJwHXEFpW7T5SZL8',
    name='transfer_to_agent',
    response={
      'result': None
    }
  )
)] role='user'
  [Event] Author: greeting_agent_stateful_v1, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={
      'user_name': 'hp'
    },
    id='call_Ddcz1YVsBHO29V90HRbl7b7q',
    name='say_hello_stateful'
  )
)] role='model'

tool_context.state['user_name']: hp
  [Event] Author: greeting_agent_stateful_v1, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='call_Ddcz1YVsBHO29V90HRbl7b7

In [17]:
# Diagnostic: Inspect which agent handles the greeting
print("Running diagnostic: verbose call for 'Hello, I'm hp!'")
await root_stateful_caller.call("Hello, I'm hp!", verbose=True)
session = await root_stateful_caller.get_session()
print("Session after hello:", session.state)

# You can also try a farewell to compare
print("Running diagnostic: verbose call for 'Thanks, bye!'")
await root_stateful_caller.call("Thanks, bye!", verbose=True)
session = await root_stateful_caller.get_session()
print("Session after farewell:", session.state)


Running diagnostic: verbose call for 'Hello, I'm hp!'

>>> User Query: Hello, I'm hp!
  [Event] Author: farewell_agent_stateful_v1, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={
      'agent_name': 'greeting_agent_stateful_v1'
    },
    id='call_N6tt5ZryXFWB9T8Lophei6tl',
    name='transfer_to_agent'
  )
)] role='model'
  [Event] Author: farewell_agent_stateful_v1, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='call_N6tt5ZryXFWB9T8Lophei6tl',
    name='transfer_to_agent',
    response={
      'result': None
    }
  )
)] role='user'
  [Event] Author: greeting_agent_stateful_v1, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={
      'user_name': 'hp'
    },
    id='call_XqUCObTFcJokwzGWtfydiQlf',
    name='say_hello_stateful'
  )
)] role='model'

tool_context.state['user_name']: hp
  [Event] Author: greeting_agent_stateful_v1, Type: Event, Final: False, Co

In [ ]:
async def run_interactive_conversation():
    while True:
        user_query = input("Ask me something (or type 'exit' to quit): ")
        if user_query.lower() == 'exit':
            break
        response = await root_stateful_caller.call(user_query)
        print(f"Response: {response}")

# Execute the interactive conversation
await run_interactive_conversation()


>>> User Query: hi

tool_context.state['user_name']: hp
<<< Agent Response: Hello to you, hp.
Response: Hello to you, hp.

>>> User Query: ssup ?

tool_context.state['user_name']: hp
<<< Agent Response: Hello to you, hp.
Response: Hello to you, hp.

>>> User Query: how to make tea?
<<< Agent Response: To make a basic cup of tea, follow these steps:

1. **Boil Water**: Bring a pot or kettle of fresh water to a boil.
2. **Pre-warm the Teacup (optional)**: Pour a little of the boiling water into your teacup to warm it up, then discard the water.
3. **Steep the Tea**:
   - If using a teabag: Place the teabag in your teacup and pour the boiling water over it.
   - If using loose leaf tea: Place the tea leaves in a tea infuser or strainer, then put it in your teacup and pour boiling water over it.
4. **Brew**: Let the tea steep. Typically, black tea steeps for 3-5 minutes, while green tea takes about 2-3 minutes.
5. **Remove Teabag/Infuser**: Once the tea has steeped to your liking, remove 